# Initiation

In [6]:
import polars as pl
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import (
    classification_report, confusion_matrix, roc_auc_score,
    accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc
)

In [9]:
# Define a function to standardize specific installation codes (station identifiers)
def ins_rep_2(df: pl.DataFrame) -> pl.DataFrame:
    """
    Replaces legacy installation codes (INST_CODE_2) with their current standardized equivalents.
    This function handles specific known mappings from old 'F_BP_*' codes to new 'F_RN_*' codes.
    This step is crucial for ensuring consistency in station identifiers across historical data,
    especially when data comes from periods before or during rebranding activities.

    Args:
        df (pl.DataFrame): Input Polars DataFrame with an 'INST_CODE_2' column.

    Returns:
        pl.DataFrame: DataFrame with the 'INST_CODE_2' column updated to standardized codes.
    """
    df = df.with_columns([
        pl.when(pl.col('INST_CODE_2') == 'F_BP_MOSCOW').then(pl.lit('F_RN_MOSCOW'))
          .when(pl.col('INST_CODE_2') == 'F_BP_PITER').then(pl.lit('F_RN_SP'))
          .when(pl.col('INST_CODE_2') == 'F_BP_TVER').then(pl.lit('F_RN_TVER'))
          .otherwise(pl.col('INST_CODE_2')) # Keep other codes as they are
          .alias('INST_CODE_2') # Overwrite the original column
    ])
    return df

# Define a function to cast a DataFrame to a predefined canonical schema
def schemas(temp2: pl.DataFrame) -> pl.DataFrame:
    """
    Casts the input DataFrame (temp2) to match the schema of a reference DataFrame (temp1).
    The reference schema is loaded from a specific Parquet file ('25.1.parquet'),
    which is assumed to represent the target canonical schema for all monthly transaction files.
    If a column from the reference schema is missing in the input DataFrame,
    it is added as a null-filled column with the correct data type.
    This ensures schema consistency when concatenating multiple monthly Parquet files
    that may have evolved schemas over time.

    Args:
        temp2 (pl.DataFrame): The input Polars DataFrame to be schema-aligned.

    Returns:
        pl.DataFrame: A new DataFrame with columns cast to the reference schema's data types.
                      Missing columns from the reference schema are added and filled with nulls.
    """
    # Load the reference schema from a master/template Parquet file
    # This file acts as the "gold standard" for the schema.
    temp1_reference_schema_df = pl.scan_parquet(fr'F:\Analytics\_1 База\2.parquets\25.1.parquet')
    
    # Perform the casting operation
    # For each column and its data type in the reference schema:
    # - If the column exists in the input DataFrame (temp2), cast it to the reference data type.
    # - If the column does not exist, create a new column with that name,
    #   fill it with nulls, and cast it to the reference data type.
    temp2_casted = temp2.select([
        pl.col(col_name).cast(target_dtype) if col_name in temp2.columns 
        else pl.lit(None).cast(target_dtype).alias(col_name)
        for col_name, target_dtype in temp1_reference_schema_df.schema.items()
    ])
    return temp2_casted

In [ ]:
# Apply schema unification and station code standardization to all loaded Parquet files.
data_test = ins_rep_2(  # Standardize 'INST_CODE_2' (station codes)
    schemas(  # Cast all loaded data to the canonical schema
        pl.scan_parquet(r'F:\Analytics\_1 База\2.parquets\*.parquet')  # Lazily scan all Parquet files in the directory
    )
)

In [ ]:
# Further refine the 'data_test' LazyFrame by applying filters, deriving new columns, and selecting a specific set of columns.
data_test = (
    data_test
    # Remove rows where CLIENT_ID_SAP == 0 (empty/invalid identifier error)
    .filter(pl.col('CLIENT_ID_SAP') != 0)
    
    # Recompute or ensure the 'SUMMA' (total transaction amount) column is accurate.
    # This is calculated as PRICE * COUNT, guarding against potential discrepancies
    # in the source data or ensuring its presence if it was missing.
    .with_columns((pl.col('PRICE') * pl.col('COUNT')).alias('SUMMA'))
    
    # Select a predefined list of relevant columns for the subsequent analysis.
    # This step helps in reducing the memory footprint and focusing on the necessary data fields.
    # Columns related to customer identity, transaction details, product information, dates,
    # and loyalty interactions are retained.
    .select([
        "EMAIL",                    # Customer email presence flag
        "SMS",                      # Customer SMS consent flag
        "IS_PETROL",                # Flag indicating if the item is petrol/fuel
        "COUNT",                    # Quantity of item/liters of fuel
        "PRICE",                    # Price per unit/liter
        "SUMMA",                    # Total amount for the transaction item (recomputed)
        "BONUS_TO_USAGE",           # Loyalty points redeemed
        "BONUS_TO_CHARGE",          # Loyalty points earned
        "PROM_BONUS_TO_CHARGE",     # Promotional loyalty points earned
        "PROMOTION_ID",             # Identifier for any promotion applied
        "TOVAR",                    # Product name/description (categorical)
        "TOVAR_2",                  # Secondary product category (categorical)
        "TOVAR_ID_GROUP",           # Product group ID
        "DATE_INT",                 # Transaction date as integer (e.g., YYYYMMDD)
        "DATE_ACTIVATION",          # Loyalty account activation date
        "DATE_FIRST",               # Customer's first transaction date
        "CRD_TYPE_2",               # Loyalty card type (e.g., PLS, VKL)
        "TOVAR_ID",                 # Specific product ID
        "MERCHANT_ID",              # Fuel station ID
        "TRANS_ID",                 # Unique transaction identifier
        "CLIENT_ID_SAP",            # Unique customer identifier
        "DATE_YM",                  # Year and month of the transaction (e.g., YYYY_MM)
        "INST_CODE_2"               # Regional ID
    ])
)
# The 'data_test' LazyFrame now incorporates these filtering and selection operations.

In [ ]:
# Encode string-based loyalty tiers in the 'LEVEL' column to ordinal integers.
# Example: 'Серебро' -> 1, 'Золото' -> 2, etc. Cast to UInt8.
data_test = data_test.with_columns(
    pl.when(pl.col('LEVEL') == 'Серебро').then(1)
    .when(pl.col('LEVEL') == 'Золото').then(2)
    .when(pl.col('LEVEL') == 'Платина').then(3)
    .when(pl.col('LEVEL') == 'Бриллиант').then(4)
    .otherwise(pl.col('LEVEL')) # Preserve original if no match;
    .alias('LEVEL')            # Overwrites original 'LEVEL' column
    .cast(pl.UInt8)
)

# Encode card types from 'CRD_TYPE_2' into a new binary numerical column 'HAS_VKL'.
# 'PLS' (Plastic) -> 0, 'VKL' (Virtual) -> 1. Cast to UInt8.
# This creates a separate feature for card type, preserving the 'LEVEL' column for loyalty tiers.
data_test = data_test.with_columns(
    pl.when(pl.col('CRD_TYPE_2') == 'PLS').then(0)
    .when(pl.col('CRD_TYPE_2') == 'VKL').then(1)
    .otherwise(None) # Assign null if CRD_TYPE_2 is neither PLS nor VKL (or handle as needed)
    .alias('HAS_VKL') # New column name to store encoded card type
    .cast(pl.UInt8)
)

In [ ]:
# Group by CLIENT_ID_SAP to find the minimum DATE_INT for each client
client_min_dates = data_test.group_by("CLIENT_ID_SAP").agg(
    pl.col("DATE_INT").min().alias("MIN_DATE_INT"),
    pl.col("DATE_FIRST").first().alias("FIRST_DATE")
)

# Join the original data with the min dates
data_with_min_dates = data_test.join(
    client_min_dates,
    on="CLIENT_ID_SAP",
    how="left"
)

In [13]:
data_test

EMAIL,SMS,IS_PETROL,COUNT,PRICE,SUMMA,BONUS_TO_USAGE,BONUS_TO_CHARGE,PROM_BONUS_TO_CHARGE,PROMOTION_ID,TOVAR,TOVAR_2,TOVAR_ID_GROUP,DATE_INT,DATE_ACTIVATION,DATE_FIRST,CRD_TYPE_2,TOVAR_ID,MERCHANT_ID,TRANS_ID,CLIENT_ID_SAP,DATE_YM,LEVEL
i8,i8,u8,f64,f64,f64,f64,f64,f64,u32,str,str,u32,i64,u32,u32,str,u32,str,u64,u32,str,u8
1,1,1,33.33,45.0,1499.85,0.0,16.67,0.0,0,"""AI""","""AI-92""",null,20220116,20220104,20220116,"""VKL""",0,"""F_RN_KALUGA_5""",100064054855,7701238490,"""2022_01""",1
1,1,1,44.24,51.0,2256.24,0.0,22.12,0.0,0,"""DT""","""DT""",null,20220128,20220128,20220128,"""PLS""",0,"""F_RN_KALUGA_25""",100072160545,8807650364,"""2022_01""",0
1,1,0,1.0,65.0,65.0,0.0,3.25,0.0,0,"""ST""","""ST""",null,20220103,20220103,20220103,"""VKL""",0,"""F_RN_KALUGA_8""",100056027900,7703974017,"""2022_01""",1
1,1,1,36.7,48.55,1781.785,0.0,18.35,0.0,0,"""AI""","""AI-95""",null,20220106,20211120,20220106,"""VKL""",0,"""F_RN_KALUGA_79""",100057813440,8803313463,"""2022_01""",1
1,1,1,11.11,45.0,499.95,0.0,5.56,0.0,0,"""AI""","""AI-92""",null,20220128,20220127,20220128,"""PLS""",0,"""F_RN_KALUGA_14""",100072680462,7700050151,"""2022_01""",0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
1,1,1,17.07,58.55,999.4485,0.0,8.54,0.0,0,"""AI""","""AI-95""",912,20250109,20230504,20230526,"""VKL""",109787,"""F_RN_KALUGA_ERP_2""",100685666615,8811045706,"""2025_01""",1
1,1,1,26.29,57.05,1499.8445,0.0,32.86,0.0,0,"""AI""","""AI-95""",908,20250125,20220128,20220204,"""PLS""",125949,"""F_RN_KALUGA_ERP_1""",100695475535,7700116054,"""2025_01""",0
1,1,1,28.14,53.3,1499.862,0.0,35.18,0.0,0,"""AI""","""AI-92""",906,20250102,20241115,20241120,"""VKL""",107305,"""F_RN_KALUGA_ERP_1""",100682115840,8814320477,"""2025_01""",1


In [14]:
def fill_missing_values(df: pl.DataFrame) -> pl.DataFrame:
    # Create a list of new column expressions with missing values filled
    new_columns = []
    for col in df.columns:
        dtype = df[col].dtype
        if isinstance(dtype, (pl.Int8, pl.Int16, pl.Int32, pl.Int64,
                              pl.UInt8, pl.UInt16, pl.UInt32, pl.UInt64,
                              pl.Float32, pl.Float64)):
            # Fill nulls with 0 for numeric columns
            new_columns.append(pl.col(col).fill_null(0))
        elif dtype == pl.Utf8:
            # Fill nulls with "EMPTY" for string columns
            new_columns.append(pl.col(col).fill_null("EMPTY"))
        else:
            # Leave other types unchanged
            new_columns.append(pl.col(col))
    # Return the DataFrame with updated columns
    return df.with_columns(new_columns)

# Apply the missing-value fill function
df = fill_missing_values(data_test)
df

EMAIL,SMS,IS_PETROL,COUNT,PRICE,SUMMA,BONUS_TO_USAGE,BONUS_TO_CHARGE,PROM_BONUS_TO_CHARGE,PROMOTION_ID,TOVAR,TOVAR_2,TOVAR_ID_GROUP,DATE_INT,DATE_ACTIVATION,DATE_FIRST,CRD_TYPE_2,TOVAR_ID,MERCHANT_ID,TRANS_ID,CLIENT_ID_SAP,DATE_YM,LEVEL
i8,i8,u8,f64,f64,f64,f64,f64,f64,u32,str,str,u32,i64,u32,u32,str,u32,str,u64,u32,str,u8
1,1,1,33.33,45.0,1499.85,0.0,16.67,0.0,0,"""AI""","""AI-92""",0,20220116,20220104,20220116,"""VKL""",0,"""F_RN_KALUGA_5""",100064054855,7701238490,"""2022_01""",1
1,1,1,44.24,51.0,2256.24,0.0,22.12,0.0,0,"""DT""","""DT""",0,20220128,20220128,20220128,"""PLS""",0,"""F_RN_KALUGA_25""",100072160545,8807650364,"""2022_01""",0
1,1,0,1.0,65.0,65.0,0.0,3.25,0.0,0,"""ST""","""ST""",0,20220103,20220103,20220103,"""VKL""",0,"""F_RN_KALUGA_8""",100056027900,7703974017,"""2022_01""",1
1,1,1,36.7,48.55,1781.785,0.0,18.35,0.0,0,"""AI""","""AI-95""",0,20220106,20211120,20220106,"""VKL""",0,"""F_RN_KALUGA_79""",100057813440,8803313463,"""2022_01""",1
1,1,1,11.11,45.0,499.95,0.0,5.56,0.0,0,"""AI""","""AI-92""",0,20220128,20220127,20220128,"""PLS""",0,"""F_RN_KALUGA_14""",100072680462,7700050151,"""2022_01""",0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
1,1,1,17.07,58.55,999.4485,0.0,8.54,0.0,0,"""AI""","""AI-95""",912,20250109,20230504,20230526,"""VKL""",109787,"""F_RN_KALUGA_ERP_2""",100685666615,8811045706,"""2025_01""",1
1,1,1,26.29,57.05,1499.8445,0.0,32.86,0.0,0,"""AI""","""AI-95""",908,20250125,20220128,20220204,"""PLS""",125949,"""F_RN_KALUGA_ERP_1""",100695475535,7700116054,"""2025_01""",0
1,1,1,28.14,53.3,1499.862,0.0,35.18,0.0,0,"""AI""","""AI-92""",906,20250102,20241115,20241120,"""VKL""",107305,"""F_RN_KALUGA_ERP_1""",100682115840,8814320477,"""2025_01""",1


In [15]:
# Define a global cutoff date for end-of-period churn assessment.
# Example: January 31, 2025. This date is used to check if customers
# have been inactive for too long by this specific point in time.
cutoff_date = pl.lit("20250131").str.strptime(pl.Date, format="%Y%m%d") # String in YYYYMMDD format parsed to Date

# Convert various string/integer date representations in the DataFrame to Polars Date type.
# This standardization is essential for consistent date arithmetic, comparisons, and sorting.
# It is assumed that 'DATE_INT', 'DATE_FIRST', and 'DATE_ACTIVATION' columns exist in 'df'
# and are in a parseable format. 'DATE_YM' is also processed.
df_temp = df.with_columns([
    pl.col("DATE_INT").cast(pl.Utf8) # Ensure string format before parsing
        .str.strptime(pl.Date, format="%Y%m%d") # Parse YYYYMMDD
        .alias("DATE_date"), # Standardized transaction date column
    pl.col("DATE_FIRST").cast(pl.Utf8)
        .str.strptime(pl.Date, format="%Y%m%d")
        .alias("DATE_FIRST_date"), # Standardized first transaction date
    pl.col("DATE_ACTIVATION").cast(pl.Utf8)
        .str.strptime(pl.Date, format="%Y%m%d", strict=False) # Parse YYYYMMDD, non-strict
        .alias("DATE_ACTIVATION_date"), # Standardized activation date
    pl.col("DATE_YM").str.replace("_", "") # Remove underscore (e.g., "2022_05" -> "202205")
        .str.strptime(pl.Date, format="%Y%m", strict=False) # Parse YYYYMM, non-strict
        .dt.month_start() # Standardize to the first day of the month
        .alias("DATE_YM_date") # Standardized year-month date
])

# Calculate the time difference (gap) in days between consecutive transactions for each customer.
# This requires sorting the data by customer and transaction date.
df_sorted = df_temp.sort(["CLIENT_ID_SAP", "DATE_date"])
df_with_diff = df_sorted.with_columns(
    pl.col("DATE_date").diff().over("CLIENT_ID_SAP").alias("date_diff") # Calculate difference within each customer group
)

# Identify the first instance where a customer has an inactivity gap exceeding 30 days.
# This 30-day threshold is the business rule for defining historical behavioral churn.
rows_with_gaps = df_with_diff.filter(
    pl.col("date_diff").dt.total_days() > 30 # Filter for gaps exceeding 30 days
)
# For customers with such gaps, determine the 'gap_cutoff_date'.
# This date marks the end of their "active" period before the churn-defining inactivity.
# Transactions after this date for these specific customers are considered post-churn for labeling.
client_gap_dates = rows_with_gaps.group_by("CLIENT_ID_SAP").agg(
    (pl.col("DATE_date") - pl.duration(days=1)).first().alias("gap_cutoff_date") # The date of the transaction that broke the silence, minus one day.
)

# Join the gap cutoff information back to the main temporary DataFrame.
df_temp = df_temp.join(client_gap_dates, on="CLIENT_ID_SAP", how="left")

# Filter out transactions that occurred after a customer's identified 'gap_cutoff_date'.
# This step truncates the transaction history for customers who exhibited historical behavioral churn,
# ensuring that features are calculated based on pre-churn activity only.
df_temp = df_temp.filter(
    pl.col("gap_cutoff_date").is_null() | (pl.col("DATE_date") <= pl.col("gap_cutoff_date"))
)

# Determine the 'DATE_LAST_date' (last observed "active" transaction date) for each customer *after* the above filtering.
df_temp = df_temp.with_columns(
    pl.max("DATE_date").over("CLIENT_ID_SAP").alias("DATE_LAST_date")
)

# Define the churn status (CHURN = 1 for churned, 0 for active) for each customer.
# A customer is labeled as churned if:
#   1. They had an internal inactivity gap exceeding 30 days during their recorded history (`gap_cutoff_date` is not null).
#   OR
#   2. Their last "active" transaction (`DATE_LAST_date`) was 30 or more days before the global `cutoff_date`.
# This logic captures both historical behavioral churn and end-of-period inactivity churn based on the 30-day rule.
df_temp = df_temp.with_columns(
    pl.when(
         pl.col("gap_cutoff_date").is_not_null() | ((cutoff_date - pl.col("DATE_LAST_date")).dt.total_days() >= 30)
    ).then(pl.lit(1))  # Mark as churned
     .otherwise(pl.lit(0)) # Mark as active
     .alias("CHURN")
)

# Remove the temporary 'gap_cutoff_date' column as it's no longer needed for the final dataset.
df_temp = df_temp.drop("gap_cutoff_date")

# Assign the processed temporary DataFrame, now containing the CHURN label and adjusted histories, back to 'df'.
df = df_temp

In [ ]:
# Add flag for new customers: 
# 1 if the total observed duration of activity (last transaction - first transaction) is less than 90 days.
df = df.with_columns([
    pl.when(
        (pl.col("DATE_LAST_date") - pl.col("DATE_FIRST_date"))
        .dt.total_days() < 90
    )
    .then(pl.lit(1))
    .otherwise(pl.lit(0))
    .alias("IS_NEW_CUSTOMER")
])

# Add flag for junior customers:
# 1 if the total observed duration of activity is between 90 and 179 days.
df = df.with_columns([
    pl.when(
        ((pl.col("DATE_LAST_date") - pl.col("DATE_FIRST_date"))
        .dt.total_days() >= 90)
    &   ((pl.col("DATE_LAST_date") - pl.col("DATE_FIRST_date"))
        .dt.total_days() < 180)
    )
    .then(pl.lit(1))
    .otherwise(pl.lit(0))
    .alias("IS_JUNIOR_CUSTOMER")
])

# Add flag for middle customers:
# 1 if the total observed duration of activity is between 180 and 364 days.
df = df.with_columns([
    pl.when(
        ((pl.col("DATE_LAST_date") - pl.col("DATE_FIRST_date"))
        .dt.total_days() >= 180)
    &   ((pl.col("DATE_LAST_date") - pl.col("DATE_FIRST_date"))
        .dt.total_days() < 365)
    )
    .then(pl.lit(1))
    .otherwise(pl.lit(0))
    .alias("IS_MIDDLE_CUSTOMER")
])

# Add flag for senior customers:
# 1 if the total observed duration of activity is 365 days or more.
df = df.with_columns([
    pl.when(
        (pl.col("DATE_LAST_date") - pl.col("DATE_FIRST_date"))
        .dt.total_days() >= 365
    )
    .then(pl.lit(1))
    .otherwise(pl.lit(0))
    .alias("IS_SENIOR_CUSTOMER")
])

In [ ]:
# Compute days between each transaction date and the customer's overall last transaction date ('DATE_LAST_date').
# This helps in creating rolling time window features relative to the end of their observed activity.
df = df.with_columns([
    (pl.col("DATE_LAST_date").cast(pl.Int64) # Cast to Int64 to ensure compatibility for subtraction
     - pl.col("DATE_date").cast(pl.Int64))   # Subtraction of dates results in a Duration type
    .dt.total_days() # Convert Duration to total days as an integer
    .alias("DAYS_BEFORE_LAST_date")
])

# Assign each transaction to a non-overlapping monthly window relative to the customer's last transaction date.
# Window 1: 0–29 days before last transaction (most recent month of activity)
# Window 2: 30–59 days before last transaction
# ...
# Window 6: 150–179 days before last transaction
# Transactions older than ~6 months (180 days) before DATE_LAST_date will be assigned None for 'month_window'.
df = df.with_columns([
    pl.when(
        (pl.col("DAYS_BEFORE_LAST_date") >= 0) 
     &  (pl.col("DAYS_BEFORE_LAST_date") < 30)
    )
    .then(1) # Most recent 30-day window
    .when(
        (pl.col("DAYS_BEFORE_LAST_date") >= 30)
     &  (pl.col("DAYS_BEFORE_LAST_date") < 60)
    )
    .then(2) # Previous 30-day window
    .when(
        (pl.col("DAYS_BEFORE_LAST_date") >= 60)
     &  (pl.col("DAYS_BEFORE_LAST_date") < 90)
    )
    .then(3)
    .when(
        (pl.col("DAYS_BEFORE_LAST_date") >= 90)
     &  (pl.col("DAYS_BEFORE_LAST_date") < 120)
    )
    .then(4)
    .when(
        (pl.col("DAYS_BEFORE_LAST_date") >= 120)
     &  (pl.col("DAYS_BEFORE_LAST_date") < 150)
    )
    .then(5)
    .when(
        (pl.col("DAYS_BEFORE_LAST_date") >= 150)
     &  (pl.col("DAYS_BEFORE_LAST_date") < 180)
    )
    .then(6) # 6th 30-day window from the last transaction
    .otherwise(None) 
    .alias("month_window")
])

In [18]:
# Create a new DataFrame reference for clarity before modifying categorical columns.
df_non_cat = df

# Handle specified categorical columns by converting their string values to unique numeric codes.
# Missing values (nulls) in these categorical columns are mapped to 0.
categorical_cols = ['TOVAR', 'TOVAR_2', 'MERCHANT_ID']
for col_name_categorical in categorical_cols:
    if col_name_categorical in df_non_cat.columns:
        # Create a mapping of unique non-null values in the column to consecutive numeric codes (starting from 1).
        unique_values = df_non_cat.select(pl.col(col_name_categorical)).unique().drop_nulls().to_series().to_list()
        mapping = {val: idx + 1 for idx, val in enumerate(unique_values)} # Mapping starts from 1
        
        # Apply the mapping: replace string values with their numeric codes. Nulls are mapped to 0.
        df_non_cat = df_non_cat.with_columns([
            pl.when(pl.col(col_name_categorical).is_null())
              .then(pl.lit(0).cast(pl.UInt32)) # Map nulls to 0
              .otherwise(
                  # Use map_dict for efficient replacement.
                  # A default value (e.g., 0) is provided for any potential values not found in the mapping,
                  # though this scenario is unlikely if unique_values are derived from the column itself.
                  pl.col(col_name_categorical).map_dict(mapping, default=pl.lit(0).cast(pl.UInt32)).cast(pl.UInt32)
              )
              .alias(col_name_categorical) # Overwrite the original column
        ])
df_non_cat

EMAIL,SMS,IS_PETROL,COUNT,PRICE,SUMMA,BONUS_TO_USAGE,BONUS_TO_CHARGE,PROM_BONUS_TO_CHARGE,PROMOTION_ID,TOVAR,TOVAR_2,TOVAR_ID_GROUP,DATE_INT,DATE_ACTIVATION,DATE_FIRST,CRD_TYPE_2,TOVAR_ID,MERCHANT_ID,TRANS_ID,CLIENT_ID_SAP,DATE_YM,LEVEL,DATE_date,DATE_FIRST_date,DATE_ACTIVATION_date,DATE_YM_date,DATE_LAST_date,CHURN,IS_NEW_CUSTOMER,IS_JUNIOR_CUSTOMER,IS_MIDDLE_CUSTOMER,IS_SENIOR_CUSTOMER,DAYS_BEFORE_LAST_date,month_window
i8,i8,u8,f64,f64,f64,f64,f64,f64,u32,u32,u32,u32,i64,str,u32,str,u32,u32,u64,u32,str,u8,date,date,date,date,date,i32,i32,i32,i32,i32,i64,i32
1,1,1,33.33,45.0,1499.85,0.0,16.67,0.0,0,1,3,0,20220116,"""20220104""",20220116,"""VKL""",0,46,100064054855,7701238490,"""2022_01""",1,2022-01-16,2022-01-16,2022-01-04,2022-01-01,2025-01-19,0,0,0,0,1,1099,null
1,1,1,44.24,51.0,2256.24,0.0,22.12,0.0,0,3,4,0,20220128,"""20220128""",20220128,"""PLS""",0,22,100072160545,8807650364,"""2022_01""",0,2022-01-28,2022-01-28,2022-01-28,2022-01-01,2022-01-28,1,1,0,0,0,0,1
1,1,0,1.0,65.0,65.0,0.0,3.25,0.0,0,2,2,0,20220103,"""20220103""",20220103,"""VKL""",0,45,100056027900,7703974017,"""2022_01""",1,2022-01-03,2022-01-03,2022-01-03,2022-01-01,2022-01-07,1,1,0,0,0,4,1
1,1,1,36.7,48.55,1781.785,0.0,18.35,0.0,0,1,1,0,20220106,"""20211120""",20220106,"""VKL""",0,35,100057813440,8803313463,"""2022_01""",1,2022-01-06,2022-01-06,2021-11-20,2022-01-01,2022-01-06,1,1,0,0,0,0,1
1,1,1,11.11,45.0,499.95,0.0,5.56,0.0,0,1,3,0,20220128,"""20220127""",20220128,"""PLS""",0,19,100072680462,7700050151,"""2022_01""",0,2022-01-28,2022-01-28,2022-01-27,2022-01-01,2023-02-21,1,0,0,0,1,389,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
0,0,1,20.0,53.3,1066.0,0.0,30.0,0.0,0,1,3,906,20250126,"""20241128""",20241129,"""VKL""",107305,52,100696352895,1414447422,"""2025_01""",1,2025-01-26,2024-11-29,2024-11-28,2025-01-01,2025-01-31,0,1,0,0,0,5,1
1,1,1,26.29,57.05,1499.8445,0.0,32.86,0.0,0,1,1,908,20250125,"""20220128""",20220204,"""PLS""",125949,31,100695475535,7700116054,"""2025_01""",0,2025-01-25,2022-02-04,2022-01-28,2025-01-01,2025-01-30,0,0,0,0,1,5,1
1,1,1,28.14,53.3,1499.862,0.0,35.18,0.0,0,1,3,906,20250102,"""20241115""",20241120,"""VKL""",107305,31,100682115840,8814320477,"""2025_01""",1,2025-01-02,2024-11-20,2024-11-15,2025-01-01,2025-01-02,0,1,0,0,0,0,1


In [19]:
# Filter out new and junior customers for specific analyses or model training cohorts.
df_non_cat_3 = df_non_cat.filter(pl.col('IS_NEW_CUSTOMER') != 1) # Excludes customers with < 90 days tenure
df_non_cat_6 = df_non_cat.filter(
    (pl.col('IS_NEW_CUSTOMER') != 1) & (pl.col('IS_JUNIOR_CUSTOMER') != 1) # Excludes customers with < 180 days tenure
)

# Create parameter table for boosting

In [22]:
# This cell aggregates transaction data to create a pivot table showing the number of 
# unique transactions for each customer within each of the last 6 monthly windows.
# This is an example of creating time-series-like features for each customer.

(df_non_cat_3 # Start with data excluding newest customers (tenure >= 90 days)
 .filter(pl.col("month_window").is_not_null()) # Ensure 'month_window' is valid for pivoting
 .group_by(["CLIENT_ID_SAP", "month_window"]) # Group by customer and their relative month window
 .agg([
     pl.col("TRANS_ID").n_unique().alias("unique_transactions") # Count unique transactions in that window
 ])
 .pivot( # Transform long format (customer-month_window rows) to wide format (customer rows, month_window columns)
     values="unique_transactions",    # Values to fill the new columns
     index="CLIENT_ID_SAP",           # Rows will be unique customers
     columns="month_window",          # New columns will be based on month_window values (1 to 6)
     aggregate_function="first"     # Since each customer-month_window combo is unique, 'first' is fine
 )
 .rename({ # Rename pivoted columns for clarity
     "1": "1m_unique_transactions", # Most recent month window
     "2": "2m_unique_transactions", 
     "3": "3m_unique_transactions",
     "4": "4m_unique_transactions",
     "5": "5m_unique_transactions",
     "6": "6m_unique_transactions"  # Oldest of the 6 windows
 })
 .with_columns([ # Fill any resulting nulls (e.g., if a customer had no activity in a window) with 0
     pl.col("1m_unique_transactions").fill_null(0),
     pl.col("2m_unique_transactions").fill_null(0),
     pl.col("3m_unique_transactions").fill_null(0),
     pl.col("4m_unique_transactions").fill_null(0),
     pl.col("5m_unique_transactions").fill_null(0),
     pl.col("6m_unique_transactions").fill_null(0)
 ])
)
# The result of this cell (if assigned to a variable or displayed) would be a DataFrame where
# each row is a customer and columns represent their unique transaction counts for each of the
# last 6 thirty-day windows relative to their last transaction.
# This type of pivoted data can be useful for models that expect features in a wide format
# or for direct input into time-series analysis if further transformed

C:\Users\kostinkd\AppData\Local\Temp\ipykernel_24888\267905750.py:7: DeprecationWarning: The argument `columns` for `DataFrame.pivot` is deprecated. It has been renamed to `on`.
  .pivot(


CLIENT_ID_SAP,2m_unique_transactions,3m_unique_transactions,1m_unique_transactions,6m_unique_transactions,4m_unique_transactions,5m_unique_transactions
u32,u32,u32,u32,u32,u32,u32
7711094587,12,5,14,5,3,0
7711291765,0,1,2,2,2,3
8811212316,2,1,2,0,0,1
8810132695,9,15,5,17,15,17
7712801667,2,0,1,2,2,1
…,…,…,…,…,…,…
9932735261,0,0,2,0,1,0
8808793996,0,0,1,0,1,0
7708633100,0,0,3,0,0,0


In [23]:
def create_ml_features_with_monthly_trends(df: pl.DataFrame) -> pl.DataFrame:
    """
    Create features for ML model including monthly behavior trends.
    Explicitly handles:
    - New customers with limited history
    - Feature normalization
    - Missing months/sporadic activity
    Note: Updated to use the new 'month_window' column for monthly metrics.
    """
    
    # First create transaction-level aggregations (or rather, aggregations per unique transaction ID for each customer)
    trans_level = df.group_by(["CLIENT_ID_SAP", "TRANS_ID"]).agg(
        pl.col("SUMMA").sum().alias("TRANSACTION_TOTAL"),
        pl.col("IS_PETROL").max().alias("HAS_FUEL_ITEMS"),
        pl.col("IS_PETROL").min().eq(0).alias("HAS_NON_FUEL_ITEMS"),
        pl.col("DATE_date").first().alias("TRANSACTION_DATE"),
        pl.col("month_window").first().alias("MONTH_WINDOW"),  # using new month_window metric
        pl.col("COUNT").filter(pl.col("IS_PETROL") == 1).sum().alias("FUEL_LITERS_IN_TRANSACTION")
    )
    
    # Monthly aggregations at the customer level using the new month_window column
    monthly_stats = (
        trans_level
        .filter(pl.col("MONTH_WINDOW").is_not_null())
        .group_by(["CLIENT_ID_SAP", "MONTH_WINDOW"])
        .agg([
            pl.len().alias("TRANSACTIONS_IN_MONTH"),  # aggregation based on month_window
            pl.col("TRANSACTION_TOTAL").sum().alias("MONTHLY_SPENT"),
            pl.col("TRANSACTION_TOTAL").mean().alias("MONTHLY_AVG_TRANSACTION"),
            pl.col("HAS_FUEL_ITEMS").sum().alias("MONTHLY_FUEL_TRANSACTIONS"),
            pl.col("HAS_NON_FUEL_ITEMS").sum().alias("MONTHLY_NON_FUEL_TRANSACTIONS"),
            pl.col("FUEL_LITERS_IN_TRANSACTION").sum().alias("MONTHLY_FUEL_LITERS")
        ])
    )
    
    # Get base customer information (non-changing attributes)
    customer_base = df.group_by("CLIENT_ID_SAP").agg(
        pl.col("DATE_LAST_date").first().alias("LATEST_TRANSACTION_DATE"),
        pl.col("DATE_ACTIVATION_date").first().alias("ACTIVATION_DATE"), 
        pl.col("EMAIL").max().alias("HAS_EMAIL"),
        pl.col("SMS").max().alias("HAS_SMS"),
        pl.col("CRD_TYPE_2").eq("VKL").any().cast(pl.Int8).alias("HAS_VKL"),
        pl.col("CHURN").first().alias("CHURN")
    )
    
    # Calculate monthly trends for each customer using explicit window functions
    monthly_trends = monthly_stats.with_columns([
        (pl.col("TRANSACTIONS_IN_MONTH") - pl.col("TRANSACTIONS_IN_MONTH").shift(1).over("CLIENT_ID_SAP")).alias("TRANSACTION_COUNT_CHANGE"),
        (pl.col("MONTHLY_SPENT") - pl.col("MONTHLY_SPENT").shift(1).over("CLIENT_ID_SAP")).alias("MONTHLY_SPENT_CHANGE"),
        (pl.col("MONTHLY_AVG_TRANSACTION") - pl.col("MONTHLY_AVG_TRANSACTION").shift(1).over("CLIENT_ID_SAP")).alias("AVG_TRANSACTION_CHANGE")
    ])
    
    # Compute month gaps for missing months detection using the numeric month_window
    monthly_trends = monthly_trends.with_columns([
         pl.col("MONTH_WINDOW").shift(1).over("CLIENT_ID_SAP").alias("PREV_MONTH_WINDOW")
    ]).with_columns([
         (pl.col("MONTH_WINDOW") - pl.col("PREV_MONTH_WINDOW")).alias("MONTHS_SINCE_LAST_ACTIVITY_INTERVAL")
    ])
    
    # Get total stats for each customer
    total_stats = trans_level.group_by("CLIENT_ID_SAP").agg(
        pl.len().alias("TOTAL_TRANSACTIONS"),
        pl.col("TRANSACTION_TOTAL").sum().alias("TOTAL_SPENT"),
        pl.col("TRANSACTION_TOTAL").mean().alias("AVG_TRANSACTION_VALUE"),
        pl.col("HAS_FUEL_ITEMS").sum().alias("FUEL_TRANSACTIONS_COUNT"),
        pl.col("HAS_NON_FUEL_ITEMS").sum().alias("NON_FUEL_TRANSACTIONS_COUNT"),
        pl.col("FUEL_LITERS_IN_TRANSACTION").sum().alias("TOTAL_FUEL_LITERS"),
        pl.col("TRANSACTION_DATE").max().alias("LAST_TRANSACTION_DATE"),
        pl.col("TRANSACTION_DATE").min().alias("FIRST_TRANSACTION_DATE")
    )
    
    # Get monthly metrics aggregated from monthly_stats
    monthly_metrics = monthly_stats.group_by("CLIENT_ID_SAP").agg(
        # For month_window, lower numbers indicate more recent activity
        pl.col("MONTH_WINDOW").min().alias("LAST_ACTIVE_MONTH"),
        pl.col("MONTH_WINDOW").max().alias("FIRST_ACTIVE_MONTH"),
        pl.col("MONTH_WINDOW").count().alias("ACTIVE_MONTHS"),
        pl.col("TRANSACTIONS_IN_MONTH").std().alias("TRANSACTION_COUNT_VOLATILITY"),
        pl.col("TRANSACTIONS_IN_MONTH").mean().alias("AVG_MONTHLY_TRANSACTIONS")
    )
    
    # Get last month stats based on the most recent month window (i.e., lowest value)
    last_month_stats = monthly_stats.join(
        monthly_metrics.select(["CLIENT_ID_SAP", "LAST_ACTIVE_MONTH"]),
        on="CLIENT_ID_SAP"
    ).filter(
        pl.col("MONTH_WINDOW") == pl.col("LAST_ACTIVE_MONTH")
    ).select([
        "CLIENT_ID_SAP",
        pl.col("TRANSACTIONS_IN_MONTH").alias("LAST_MONTH_TRANSACTIONS"),
        pl.col("MONTHLY_SPENT").alias("LAST_MONTH_SPENT"),
        pl.col("MONTHLY_FUEL_TRANSACTIONS").alias("LAST_MONTH_FUEL_TRANS")
    ])
    
    # Get recent transaction data (last 3 months = 90 days from last transaction date)
    recent_transactions = trans_level.join(
        total_stats.select(["CLIENT_ID_SAP", "LAST_TRANSACTION_DATE"]),
        on="CLIENT_ID_SAP"
    ).filter(
        pl.col("TRANSACTION_DATE") >= pl.col("LAST_TRANSACTION_DATE") - pl.duration(days=90)
    )
    
    # Calculate 3-month averages from recent transactions
    three_month_avgs = recent_transactions.group_by("CLIENT_ID_SAP").agg(
        pl.len().alias("LAST_3M_TRANSACTION_COUNT"),
        pl.col("TRANSACTION_TOTAL").sum().alias("LAST_3M_TOTAL_SPENT"),
        pl.col("TRANSACTION_TOTAL").mean().alias("LAST_3M_AVG_TRANSACTION")
    ).with_columns([
        (pl.col("LAST_3M_TRANSACTION_COUNT") / 3).alias("LAST_3M_AVG_TRANSACTIONS"),
        (pl.col("LAST_3M_TOTAL_SPENT") / 3).alias("LAST_3M_AVG_SPENT")
    ])
    
    # NOTE: Six-month metrics removed as per instruction (шестимесячные показатели не считаем)
    
    # Calculate trend metrics from recent monthly trends
    # For recent trends, consider month windows <= 3 (i.e., last 3 months)
    recent_monthly_trends = monthly_trends.filter(pl.col("MONTH_WINDOW") <= 3)
    trend_metrics = recent_monthly_trends.group_by("CLIENT_ID_SAP").agg(
        pl.col("TRANSACTION_COUNT_CHANGE").mean().alias("RECENT_TRANSACTION_TREND"),
        pl.col("MONTHLY_SPENT_CHANGE").mean().alias("RECENT_SPENDING_TREND")
    )
    
    # Calculate gap metrics from the last 6 month windows (i.e., month_window <= 6)
    recent_gap_metrics = monthly_trends.filter(pl.col("MONTH_WINDOW") <= 6)
    gap_metrics = recent_gap_metrics.group_by("CLIENT_ID_SAP").agg(
        pl.col("MONTHS_SINCE_LAST_ACTIVITY_INTERVAL").mean().alias("AVG_GAP_BETWEEN_ACTIVE_MONTHS"),
        pl.col("MONTHS_SINCE_LAST_ACTIVITY_INTERVAL").max().alias("MAX_GAP_BETWEEN_ACTIVE_MONTHS")
    )
    
    # Combine all the metrics
    features_df = customer_base.join(total_stats, on="CLIENT_ID_SAP")
    features_df = features_df.join(monthly_metrics, on="CLIENT_ID_SAP", how="left")
    features_df = features_df.join(last_month_stats, on="CLIENT_ID_SAP", how="left")
    features_df = features_df.join(three_month_avgs, on="CLIENT_ID_SAP", how="left")
    features_df = features_df.join(trend_metrics, on="CLIENT_ID_SAP", how="left")
    features_df = features_df.join(gap_metrics, on="CLIENT_ID_SAP", how="left")
    
    # Handle NULL values in trend and gap metrics by setting appropriate default values
    features_df = features_df.with_columns([
        pl.col("RECENT_TRANSACTION_TREND").fill_null(0).alias("RECENT_TRANSACTION_TREND"),
        pl.col("RECENT_SPENDING_TREND").fill_null(0).alias("RECENT_SPENDING_TREND"),
        pl.col("AVG_GAP_BETWEEN_ACTIVE_MONTHS").fill_null(1).alias("AVG_GAP_BETWEEN_ACTIVE_MONTHS"),
        pl.col("MAX_GAP_BETWEEN_ACTIVE_MONTHS").fill_null(1).alias("MAX_GAP_BETWEEN_ACTIVE_MONTHS")
    ])
    
    # Calculate reference date for recent activity measures using the max date from the dataset
    max_date = df["DATE_date"].max()
    
    # Add derived features including ratios and normalized trends
    features_df = features_df.with_columns([
        # Calculate customer tenure in days
        (pl.col("LATEST_TRANSACTION_DATE") - pl.col("ACTIVATION_DATE")).dt.total_days().alias("CUSTOMER_TENURE_DAYS")
    ])
    
    # Add months since last activity (approximated as 30 days per month)
    features_df = features_df.with_columns([
        ((max_date - pl.col("LAST_TRANSACTION_DATE")) / pl.duration(days=30)).cast(pl.Int32).alias("MONTHS_SINCE_LAST_ACTIVITY")
    ])
    
    # Add activity density: ratio of active months to total months period
    features_df = features_df.with_columns([
        (pl.col("ACTIVE_MONTHS") / 
         (((pl.col("LAST_TRANSACTION_DATE") - pl.col("FIRST_TRANSACTION_DATE")) / pl.duration(days=30)).cast(pl.Int32) + 1).clip(lower_bound=1)
        ).alias("MONTHLY_ACTIVITY_DENSITY")
    ])
    
    # Add recent vs historical activity and spending metrics
    features_df = features_df.with_columns([
        (pl.col("LAST_3M_AVG_TRANSACTIONS") / pl.col("AVG_MONTHLY_TRANSACTIONS").clip(lower_bound=0.1)).alias("RECENT_VS_HISTORICAL_ACTIVITY"),
        (pl.col("LAST_3M_AVG_SPENT") / (pl.col("TOTAL_SPENT") / pl.col("ACTIVE_MONTHS")).clip(lower_bound=0.1)).alias("RECENT_VS_HISTORICAL_SPENDING")
    ])
    
    # Add normalized trends as percentage changes
    features_df = features_df.with_columns([
        (pl.col("RECENT_TRANSACTION_TREND") / pl.col("LAST_3M_AVG_TRANSACTIONS").clip(lower_bound=1) * 100).alias("PERCENT_TRANSACTION_TREND"),
        (pl.col("RECENT_SPENDING_TREND") / pl.col("LAST_3M_AVG_SPENT").clip(lower_bound=1) * 100).alias("PERCENT_SPENDING_TREND")
    ])
    
    # Add customer tenure flags
    features_df = features_df.with_columns([
        # Flag for new customers (less than 3 months of activity)
        (pl.col("ACTIVE_MONTHS") < 3).cast(pl.Int8).alias("IS_NEW_CUSTOMER"),
        # Flag for mid-tenure customers (3-12 months)
        ((pl.col("ACTIVE_MONTHS") >= 3) & (pl.col("ACTIVE_MONTHS") < 12)).cast(pl.Int8).alias("IS_MID_TENURE_CUSTOMER"),
        # Flag for established customers (12+ months)
        (pl.col("ACTIVE_MONTHS") >= 12).cast(pl.Int8).alias("IS_ESTABLISHED_CUSTOMER"),
        # Confidence measure for trend reliability based on history length
        (pl.col("ACTIVE_MONTHS").clip(lower_bound=1, upper_bound=12) / 12).alias("TREND_RELIABILITY")
    ])
    
    # Add normalization metrics for volatility and transactions z-score
    features_df = features_df.with_columns([
        (pl.col("TRANSACTION_COUNT_VOLATILITY") / pl.col("AVG_MONTHLY_TRANSACTIONS").clip(lower_bound=1)).alias("NORMALIZED_VOLATILITY"),
        ((pl.col("LAST_MONTH_TRANSACTIONS") - pl.col("AVG_MONTHLY_TRANSACTIONS")) / pl.col("TRANSACTION_COUNT_VOLATILITY").clip(lower_bound=1)).alias("LAST_MONTH_TRANSACTIONS_Z_SCORE")
    ])
    
    # Add missing months handling flags and inactivity period metrics
    features_df = features_df.with_columns([
        (pl.col("AVG_GAP_BETWEEN_ACTIVE_MONTHS") > 1).cast(pl.Int8).alias("HAS_IRREGULAR_ACTIVITY"),
        (pl.col("MONTHS_SINCE_LAST_ACTIVITY") > 2).cast(pl.Int8).alias("HAS_RECENT_INACTIVITY"),
        (pl.col("MONTHLY_ACTIVITY_DENSITY") >= 0.75).cast(pl.Int8).alias("IS_CONSISTENT_CUSTOMER"),
        (pl.col("MAX_GAP_BETWEEN_ACTIVE_MONTHS").clip(lower_bound=1, upper_bound=6)).alias("MAX_RECENT_INACTIVITY_PERIOD")
    ])
    
    # Add fuel station specific metrics
    features_df = features_df.with_columns([
        (pl.col("FUEL_TRANSACTIONS_COUNT") / pl.col("TOTAL_TRANSACTIONS") * 100).alias("FUEL_TRANSACTION_PERCENTAGE"),
        (pl.col("NON_FUEL_TRANSACTIONS_COUNT") / pl.col("TOTAL_TRANSACTIONS") * 100).alias("NON_FUEL_TRANSACTION_PERCENTAGE"),
        (pl.col("TOTAL_FUEL_LITERS") / pl.col("FUEL_TRANSACTIONS_COUNT").clip(lower_bound=1)).alias("AVG_FUEL_LITERS_PER_TRANSACTION"),
        (pl.col("LAST_MONTH_FUEL_TRANS") / pl.col("FUEL_TRANSACTIONS_COUNT").clip(lower_bound=1) * pl.col("ACTIVE_MONTHS")).alias("RECENT_FUEL_PURCHASE_TREND")
    ])
    
    return features_df

# Features calculation

In [24]:
# Generate ML features using customers with an observed activity span of at least 90 days.
churn_features_df = create_ml_features_with_monthly_trends(df_non_cat_3) # df_non_cat_3: first-to-last transaction span >= 90 days

# Further filter to retain only customers with at least 3 active months.
churn_features_df = churn_features_df.filter(pl.col("IS_NEW_CUSTOMER") != 1) # IS_NEW_CUSTOMER based on ACTIVE_MONTHS < 3 (from function)

# Final cleanup: remove rows with null activation dates and drop auxiliary date/identifier columns.
churn_features_df = churn_features_df.filter(pl.col("ACTIVATION_DATE").is_not_null())
churn_features_df = churn_features_df.drop([
    "LATEST_TRANSACTION_DATE",    # Intermediate date for recency
    "ACTIVATION_DATE",            # Raw date, tenure already derived
    "LAST_TRANSACTION_DATE",      # Raw date, recency/window features derived
    "FIRST_TRANSACTION_DATE",     # Raw date, tenure features derived
    "LAST_ACTIVE_MONTH",          # Helper for monthly aggregations
    "FIRST_ACTIVE_MONTH"          # Helper for monthly aggregations
])

# churn_features_df is now the final dataset for modeling.
churn_features_df

CLIENT_ID_SAP,HAS_EMAIL,HAS_SMS,HAS_VKL,CHURN,TOTAL_TRANSACTIONS,TOTAL_SPENT,AVG_TRANSACTION_VALUE,FUEL_TRANSACTIONS_COUNT,NON_FUEL_TRANSACTIONS_COUNT,TOTAL_FUEL_LITERS,ACTIVE_MONTHS,TRANSACTION_COUNT_VOLATILITY,AVG_MONTHLY_TRANSACTIONS,LAST_MONTH_TRANSACTIONS,LAST_MONTH_SPENT,LAST_MONTH_FUEL_TRANS,LAST_3M_TRANSACTION_COUNT,LAST_3M_TOTAL_SPENT,LAST_3M_AVG_TRANSACTION,LAST_3M_AVG_TRANSACTIONS,LAST_3M_AVG_SPENT,RECENT_TRANSACTION_TREND,RECENT_SPENDING_TREND,AVG_GAP_BETWEEN_ACTIVE_MONTHS,MAX_GAP_BETWEEN_ACTIVE_MONTHS,CUSTOMER_TENURE_DAYS,MONTHS_SINCE_LAST_ACTIVITY,MONTHLY_ACTIVITY_DENSITY,RECENT_VS_HISTORICAL_ACTIVITY,RECENT_VS_HISTORICAL_SPENDING,PERCENT_TRANSACTION_TREND,PERCENT_SPENDING_TREND,IS_NEW_CUSTOMER,IS_MID_TENURE_CUSTOMER,IS_ESTABLISHED_CUSTOMER,TREND_RELIABILITY,NORMALIZED_VOLATILITY,LAST_MONTH_TRANSACTIONS_Z_SCORE,HAS_IRREGULAR_ACTIVITY,HAS_RECENT_INACTIVITY,IS_CONSISTENT_CUSTOMER,MAX_RECENT_INACTIVITY_PERIOD,FUEL_TRANSACTION_PERCENTAGE,NON_FUEL_TRANSACTION_PERCENTAGE,AVG_FUEL_LITERS_PER_TRANSACTION,RECENT_FUEL_PURCHASE_TREND
u32,i8,i8,i8,i32,u32,f64,f64,i64,u32,f64,u32,f64,f64,u32,f64,i64,u32,f64,f64,f64,f64,f64,f64,f64,i32,i64,i32,f64,f64,f64,f64,f64,i8,i8,i8,f64,f64,f64,i8,i8,i8,i32,f64,f64,f64,f64
7713274081,1,1,1,0,35,65240.824,1864.023543,34,6,1260.23,6,0.816497,3.333333,4,8092.0,4,11,21385.75,1944.159091,3.666667,7128.583333,1.0,1136.8725,0.2,4,301,0,0.6,1.1,0.655594,27.272727,15.948085,0,1,0,0.5,0.244949,0.666667,0,0,0,4,97.142857,17.142857,37.065588,0.705882
8813338581,1,1,1,0,10,17309.75,1730.975,9,4,265.0,3,1.0,2.0,2,3786.5,2,3,4911.5,1637.166667,1.0,1637.166667,4.2950e9,-1654.5,-1.0,3,258,0,0.333333,0.5,0.283742,4.2950e11,-101.05874,0,1,0,0.25,0.5,0.0,0,0,0,3,90.0,40.0,29.444444,0.666667
7712459552,1,1,1,0,34,52074.382,1531.599471,33,1,1037.8,5,1.516575,2.6,2,3565.926,2,7,10031.541,1433.077286,2.333333,3343.847,2.0,2384.643,-0.25,5,504,0,0.294118,0.897436,0.321064,85.714286,71.314357,0,1,0,0.416667,0.583298,-0.395628,0,0,0,5,97.058824,2.941176,31.448485,0.30303
8813062265,1,1,1,0,60,66478.4995,1107.974992,60,0,1214.51,6,7.456541,6.0,21,27718.118,21,25,30976.1375,1239.0455,8.333333,10325.379167,1.4317e9,-613.681333,-0.4,3,370,0,0.461538,1.388889,0.931914,1.7180e10,-5.943427,0,1,0,0.5,1.242757,2.011657,0,0,0,3,100.0,0.0,20.241833,2.1
7708889592,1,1,1,1,12,13392.5735,1116.047792,9,3,241.31,5,0.547723,1.4,1,1999.8615,1,4,4508.429,1127.10725,1.333333,1502.809667,0.0,257.0475,-0.5,4,416,13,0.357143,0.952381,0.561061,0.0,17.104461,0,1,0,0.416667,0.39123,-0.4,0,1,0,4,75.0,25.0,26.812222,0.555556
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
8812347728,1,1,1,1,34,17666.0375,519.589338,34,0,337.77,6,2.949576,3.5,9,4230.8855,9,13,6468.5055,497.577346,4.333333,2156.1685,1.4317e9,660.6285,-0.8,3,311,7,0.545455,1.238095,0.73231,3.3038e10,30.639002,0,1,0,0.5,0.842736,1.864675,0,1,0,3,100.0,0.0,9.934412,1.588235
7712743719,1,1,1,0,50,63786.157,1275.72314,50,1,1268.3,6,1.36626,3.666667,4,5680.613,4,11,14853.1155,1350.283227,3.666667,4951.0385,1.4317e9,393.511,-0.2,4,457,0,0.375,1.0,0.465716,3.9045e10,7.94805,0,1,0,0.5,0.372616,0.243975,0,0,0,4,100.0,2.0,25.366,0.48
8811646324,0,0,1,1,10,14177.923,1417.7923,10,1,291.31,4,2.0,2.0,1,2612.203,1,2,4611.787,2305.8935,0.666667,1537.262333,0.0,590.6015,-1.333333,1,212,12,0.5,0.333333,0.433706,0.0,38.419045,0,1,0,0.333333,1.0,-0.5,0,1,0,1,100.0,10.0,29.131,0.4
